In [ ]:
import csv
import pandas as pd
import folium
import glob
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
rec_data_name = 'data/recievers.csv'
rec_data = pd.read_csv(rec_data_name)

filelist = glob.glob('data/observations/*.csv')
dfs = dict()
rec_ids = []
for f in filelist:
    rec_id = f.split('_')[1]
    dfs[rec_id]=pd.read_csv(f)
    rec_ids.append(rec_id)

In [ ]:
m = folium.Map(location=[57.0890, -4.7483], zoom_start=15)
recs = list()
for index, rec in rec_data[rec_data['system']=='Ness'].iterrows():
    recs.append(rec)
    if str(int(rec['id'])) in rec_ids:
        colour = 'red'
        df = dfs[str(int(rec['id']))]
        fishes = len(df['Transmitter'].unique())
    else:
        colour = 'black'
        fishes = 0

    folium.CircleMarker(location = [rec['lat'], rec['lon']], popup=rec['id'], 
                        color = colour,
                      radius=np.sqrt(fishes),
                      fill=True,
                      fill_color=colour,
                        stroke = True,
                        weight=1,
                        fill_opacity=.55
                     ).add_to(m)

    

m

In [ ]:
import dateutil
timethreshold = 30 #how many seconds max difference between pings from a fish
dffilts = dict()
for key, df in dfs.items():
    df['tesc'] = df.apply (lambda row: dateutil.parser.isoparse(row['Date and Time (UTC)']).timestamp(), axis=1)
    df['datetime'] = df.apply (lambda row: dateutil.parser.isoparse(row['Date and Time (UTC)']), axis=1)
    df = df [['tesc','datetime','Transmitter']]
    #df.set_index('tesc')
    df['fishdiff']= ~(df['Transmitter'] == df['Transmitter'].shift())
    df['mfishdiff']= ~(df['Transmitter'].shift(-1) == df['Transmitter'])
    df['tdiff']=df['tesc'] - df['tesc'].shift(1, fill_value=1554901439)
    df['mtdiff']=df['tesc'].shift(-1, fill_value=1554901439) - df['tesc']
    dffilt = df[((df['tdiff']<timethreshold) & (df['fishdiff'])) | ((df['mtdiff']<timethreshold)  & (df['mfishdiff']))]
    plt.plot(dffilt['datetime'],dffilt['tdiff'],'o')
    plt.show()
    display(dffilt.head(10))
    dffilts[key]=dffilt

In [ ]:
#give clusters a name
for dffilt in dffilts.values():
    groupid = 0
    shoals = []
    for ix, valx in dffilt.iterrows():
        if valx['tdiff']>=timethreshold:
            groupid=groupid+1
        shoals.append(groupid)
        
    dffilt['shoals']=shoals        
    plt.plot(dffilt['datetime'],dffilt['shoals'],'o')
    plt.show()

#plot cluster sizes x cluster number per time
#show that on a map

In [ ]:
for key, dffilt in dffilts.items():
    print(key)
    plt.plot(dffilt.groupby('shoals').size())
    plt.show()